Procesamiento de datos del segundo fomulario. El que hice en Exactas y Fadu con aprox 15 personas en cada lugar.

Los datos originales estan aca:

https://github.com/IoniUTDT/Magia/tree/master/PilotoFacuFadu

In [1]:
%load_ext autoreload
%autoreload 2
import pandas
from IPython.display import display
import sys
sys.path.append("..")
import leerCSV
data = leerCSV.leerCSV ('./Data_Exp2.csv','Codigos.txt')
data.to_csv('dataFadu.csv')

In [2]:
# data = pandas.read_csv('./Data_Exp2.csv')
display (data.head())

,questionnaire_id,global_id,Edad,Profesion,Estudios,Justificacion1,Seguridad1,JustificacionSecundaria,RelevanciaMano,Justificacion2,...,Religion,Reparar,Programar,Correr,Acertijos,Astrologia,Deportes,Debatir,Magia,Electronica
0,['31256'],1002,18-24,Mate/Comp/Sist,Secundaria,Calor,5,"[Magnetismo, Presion]",3,No Se,...,No Gusta,Ni,Gusta,Ni,Encanta,Gusta,Gusta,Encanta,Gusta,Gusta
1,['59406'],1002,18-24,Null,Null,Calor,5,[Magnetismo],2,Electricidad,...,Ni,Ni,Ni,Gusta,Gusta,Gusta,No Gusta,Ni,Gusta,Ni
2,['55654'],1002,18-24,Ingenieria,Universidad,Magnetismo,0,"[Calor, Presion, Electricidad]",4,Magnetismo,...,Odio,Encanta,Encanta,Gusta,Gusta,No Gusta,No Gusta,Gusta,No Gusta,Encanta
3,['15766'],1002,25-40,Bio/Qui/Geo/Pale,Universidad,Presion,6,[Otra],1,Presion,...,Odio,Encanta,Gusta,Gusta,Gusta,Ni,Gusta,Gusta,Gusta,Encanta
4,['61935'],1002,18-24,Bio/Qui/Geo/Pale,Null,Magnetismo,3,[Presion],4,Presion,...,No Gusta,Ni,Ni,Gusta,Gusta,Gusta,Gusta,Gusta,Gusta,Ni
